In [1]:
import sys
import seaborn as sns
import pandas as pd 
import numpy as np
from itertools import combinations
from scipy.spatial.distance import squareform, pdist
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import torch
import anndata as an
import scanpy as sc
import os
import gc
from importlib import reload

from datasets import Dataset, load_from_disk
from datasets import load_dataset
from geneformer import EmbExtractor

# classifer tools
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix


import pandas as pd 
import numpy as np
import anndata as an
import scanpy as sc
import pickle

from datasets import Dataset, load_from_disk, load_dataset
import geneformer

from datetime import datetime

DEFAULT_NAME_PATH = "/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer/gene_name_id_dict.pkl"
DEFAULT_TOKEN_PATH = "/nfs/turbo/umms-indikar/shared/projects/geneformer/token_dictionary.pkl"
DEFAULT_MEDIAN_PATH = "/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer/gene_median_dictionary.pkl"

sns.set_style('white')
torch.cuda.empty_cache()


/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Parameters v
###

# cells to filter on
initial_cell_type = 'Fibroblast'

# list of genes to perturb with at the front of the list
gene_list = [
    'GATA2', 
    'GFI1B', 
    'FOS', 
    'STAT5A',
    'REL',
    'FOSB',
    'IKZF1',
    'RUNX3',
    'MEF2C',
    'ETV6',
]

############### important! ###############
# added back sampling, now all cells
num_initial_cells = 10
################  (1/2)  #################
MODEL_PATH = "/scratch/indikar_root/indikar1/shared_data/geneformer/fine_tune/240715_geneformer_cellClassifier_no_induced/ksplit1/"
#     Args:
#         MODEL_PATH (str): Path to the model file.
#         model_type (str, optional): Type of model ('Pretrained' or custom). Default: 'Pretrained'.
#         n_classes (int, optional): Number of output classes for custom models. Default: 0.
#         mode (str, optional): Mode to load the model in ('eval' or 'train'). Default: 'eval'.
model = geneformer.perturber_utils.load_model('Pretrained', 0 , MODEL_PATH, 'eval')


print('model loaded!')


TOKEN_DATA_PATH = "/scratch/indikar_root/indikar1/shared_data/geneformer/resources/token_mapping.csv"
token_df = pd.read_csv(TOKEN_DATA_PATH)


DATA_PATH = "/scratch/indikar_root/indikar1/shared_data/geneformer/fine_tune/hsc.dataset"



###
### Parameters ^

Some weights of BertForMaskedLM were not initialized from the model checkpoint at /scratch/indikar_root/indikar1/shared_data/geneformer/fine_tune/240715_geneformer_cellClassifier_no_induced/ksplit1/ and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model loaded!


# Load In Things

In [28]:
# Uses token_df to translate from gene_list to tokens_list v
def get_tokens_list(gene_list):
    # Get a df of the genes we are perturbing with
    genes = token_df[token_df['gene_name'].isin(gene_list)]
    
    tf_map = dict(zip(genes['gene_name'].values, genes['token_id'].values))
    
    # Create tokens_list by looking up each gene_name in the tf_map
    tokens_list = [tf_map.get(gene_name, gene_name) for gene_name in gene_list]

    return tokens_list


# def load_model(MODEL_PATH, model_type='Pretrained', n_classes=0, mode='eval'):
#     """
#     Loads a pre-trained or custom model for geneformer perturbations.

#     Args:
#         MODEL_PATH (str): Path to the model file.
#         model_type (str, optional): Type of model ('Pretrained' or custom). Default: 'Pretrained'.
#         n_classes (int, optional): Number of output classes for custom models. Default: 0.
#         mode (str, optional): Mode to load the model in ('eval' or 'train'). Default: 'eval'.

#     Returns:
#         The loaded model object.
#     """

#     model = geneformer.perturber_utils.load_model(
#         model_type,
#         n_classes,
#         MODEL_PATH,
#         mode
#     )

#     return model


def add_perturbations_to_cell(cell_tokens, perturbation_tokens):
    """
    Requires: cell_tokens is a list of (in our use, 2048) integer tokens, each token representing a gene,
        perturbation_tokens is a list of integer tokens
    Effects: returns final_tokens, a list of tokens with perturbation_tokens at the front
    """
    original_length = len(cell_tokens)

    # Remove existing perturbation tokens from the cell
    cell_tokens = [token for token in cell_tokens if token not in perturbation_tokens]

    # Add perturbations, then slice or pad to match original length
    final_tokens = (perturbation_tokens + cell_tokens)[:original_length]  # Slice if too long
    final_tokens += [0] * (original_length - len(final_tokens))            # Pad if too short

    return final_tokens


def extract_embedding_in_mem(model, data, emb_mode='cell', layer_to_quant=-1, forward_batch_size=10):
    """Extracts embeddings from a model and returns them as a DataFrame.

    This function provides an in-memory extraction of embeddings, allowing for convenient
    manipulation and analysis directly within your Python environment.

    Args:
        model: The model to use for embedding extraction.
        data: The input data for which embeddings need to be extracted.
        emb_mode (str, optional): The embedding mode. Defaults to 'cell'.
        layer_to_quant (int, optional): The layer to quantize. Defaults to -1 (last layer).
        forward_batch_size (int, optional): The batch size for forward passes. Defaults to 10.

    Returns:
        pandas.DataFrame: A DataFrame containing the extracted embeddings.

    Raises:
        TypeError: If `model` is not a supported model type.
        ValueError: If `data` is not in the correct format.
    """

    embs = geneformer.emb_extractor.get_embs(
        model,
        data,
        emb_mode,
        layer_to_quant,
        0,  # Assuming this is a constant parameter for the function
        forward_batch_size,
        summary_stat=None,  
        silent=False, 
    )
    data = embs.cpu().numpy()
    if emb_mode=='cell':
        return pd.DataFrame(data)
    else:
        return data

def embedding_to_adata(df: pd.DataFrame, n_dim: int = None) -> an.AnnData:
    """Converts a Pandas DataFrame with an embedding to an AnnData object.

    Args:
        df: The input DataFrame with numerical embedding columns and optional metadata columns.
        n_dim: The number of dimensions to keep in the embedding. If None, all dimensions are kept.

    Returns:
        The converted AnnData object.

    Raises:
        ValueError: If `n_dim` exceeds the available dimensions in the DataFrame.
    """

    if n_dim is not None and n_dim > df.shape[1]:
        raise ValueError(f"n_dim ({n_dim}) exceeds available dimensions ({df.shape[1]})")

    # Assuming embedding columns are those that are not integers
    is_metadata = df.columns.astype(str).str.isdigit()
    metadata_df = df.loc[:, ~is_metadata]
    embedding_df = df.loc[:, is_metadata]

    cell_index = pd.Index([f"C{x}" for x in range(df.shape[0])], name='obs_names')

    if n_dim is not None:
        embedding_df = embedding_df.iloc[:, :n_dim]

    var_index = pd.Index([f"D{x}" for x in range(embedding_df.shape[1])], name='var_names')

    adata = an.AnnData(embedding_df.to_numpy())
    adata.obs_names = cell_index
    adata.var_names = var_index
    adata.obs = metadata_df
    return adata

"""
For us only!!!! vvv
"""
def ten_choose_five():
    gene_list = [
    'GATA2', 
    'GFI1B', 
    'FOS', 
    'STAT5A',
    'REL',
    'FOSB',
    'IKZF1',
    'RUNX3',
    'MEF2C',
    'ETV6',
]
# Define the length of sublists
    len_sublist = 5
    
    # Generate all combinations of the specified length
    sublists = list(combinations(gene_list, len_sublist))
    
    # Create the DataFrame
    df = pd.DataFrame({
        'recipe_iteration': range(1, len(sublists) + 1),
        'recipe_list': [list(sublist) for sublist in sublists]
    })
    return df

# Check and ensure all data is of correct type
def check_and_convert(data):
    if isinstance(data, pd.DataFrame):
        for col in data.columns:
            if not pd.api.types.is_string_dtype(data[col]):
                data[col] = data[col].astype(str)
    return data

    

In [4]:
### Preliminaries v
###

if torch.cuda.is_available(): 
    print("CUDA is available! Devices: ", torch.cuda.device_count()) 
    print("Current CUDA device: ", torch.cuda.current_device()) 
    print("Device name: ", torch.cuda.get_device_name(torch.cuda.current_device())) 
else: print("CUDA is not available")



###
### Preliminaries ^







    # jobNumber = int(sys.argv[1])
    # print(jobNumber)


CUDA is available! Devices:  1
Current CUDA device:  0
Device name:  NVIDIA A100 80GB PCIe MIG 3g.40gb


In [5]:
# ### Format raw data, print messages to check v
# ###

# # Load from pre-trained data
# raw_data = load_from_disk(DATA_PATH)

# # Convert to DataFrame for filtering
# df = raw_data.to_pandas()
# print("\nOriginal Dataset:")
# print(f"  - Number of samples: {df.shape[0]:,}")
# print(f"  - Number of columns: {df.shape[1]:,}")

# # Filtering
# fb_df = df[df['standardized_cell_type'] == initial_cell_type]

# ############### important! ###############
# # sampling (ADDED BACK!)
# fb_df = fb_df.sample(num_initial_cells)
# ################  (2/2)  #################

# fb_df = fb_df.reset_index(drop=True)

# # add a cell id
# fb_df['cell_id'] = [f"cell_{i+1}" for i in range(len(fb_df))]
# fb_df['recipe'] = 'raw'  # as opposed to having a speciofic ;-separated recipe list. other entries will have this.
# fb_df['type'] = 'initial' # this dataframe no longer includes 'target'

# print("\nFiltered Dataset:")
# print(f"  - Number of samples: {fb_df.shape[0]:,}")   # Nicer formatting with commas
# print(f"  - Number of columns: {fb_df.shape[1]:,}")

# # Value counts with sorting
# print("\nCell Type Distribution (Filtered):")
# print(fb_df['standardized_cell_type'].value_counts().sort_index())  # Sort for readability

# # Convert back to Dataset
# fb_data = Dataset.from_pandas(fb_df)
# print(f"\nDataset converted back: {fb_data}")

# ###
# ### Format raw data, print messages to check ^

In [6]:

# ### Perform the perturbation v
# ###

# reprogramming_df = [
#     fb_df
# ]

# perturb = fb_df.copy()
# recipe = ";".join(gene_list)
# perturb['recipe'] = recipe
# perturb['type'] = 'reprogrammed'
# perturb['input_ids'] = perturb['input_ids'].apply(lambda x: add_perturbations_to_cell(x, get_tokens_list(gene_list)))

# reprogramming_df.append(perturb)

# reprogramming_df = pd.concat(reprogramming_df, ignore_index=True)

# print(f"{reprogramming_df.shape=}")
# reprogramming_df.sample(10)

# ###
# ### Perform the perturbation ^


In [7]:

# torch.cuda.empty_cache()

# reprogramming_data = Dataset.from_pandas(reprogramming_df)

# reprogramming_embs = extract_embedding_in_mem(
#     model, 
#     reprogramming_data, 
#     layer_to_quant=-1,
#     forward_batch_size=100,
# )
# print(f"{reprogramming_embs.shape=}")

# # translate into an anndata object and plot
# reprogramming_adata = embedding_to_adata(reprogramming_embs)
# reprogramming_adata.obs = reprogramming_df.copy()

# # sc.tl.pca(reprogramming_adata, n_comps=25)
# # sc.pp.neighbors(reprogramming_adata, n_neighbors=200)
# # sc.tl.umap(reprogramming_adata, 
# #            min_dist=0.75,
# #           )

# reprogramming_adata.obs.head()





In [8]:
#reprogramming_adata.X

In [21]:
# draft of single perturbation:
def single_recipe(recipe_list):
    ### Format raw data, print messages to check v
    ###

    # Load from pre-trained data
    raw_data = load_from_disk(DATA_PATH)

    # Convert to DataFrame for filtering
    df = raw_data.to_pandas()
    print("\nOriginal Dataset:")
    print(f"  - Number of samples: {df.shape[0]:,}")
    print(f"  - Number of columns: {df.shape[1]:,}")

    # Filtering
    fb_df = df[df['standardized_cell_type'] == initial_cell_type]

    ############### important! ###############
    # sampling (ADDED BACK!)
    fb_df = fb_df.sample(num_initial_cells)
    ################  (2/2)  #################

    fb_df = fb_df.reset_index(drop=True)

    # add a cell id
    fb_df['cell_id'] = [f"cell_{i+1}" for i in range(len(fb_df))]
    fb_df['recipe'] = 'raw'  # as opposed to having a speciofic ;-separated recipe list. other entries will have this.
    fb_df['type'] = 'initial' # this dataframe no longer includes 'target'

    print("\nFiltered Dataset:")
    print(f"  - Number of samples: {fb_df.shape[0]:,}")   # Nicer formatting with commas
    print(f"  - Number of columns: {fb_df.shape[1]:,}")

    # Value counts with sorting
    print("\nCell Type Distribution (Filtered):")
    print(fb_df['standardized_cell_type'].value_counts().sort_index())  # Sort for readability

    # Convert back to Dataset
    fb_data = Dataset.from_pandas(fb_df)
    print(f"\nDataset converted back: {fb_data}")

    ###
    ### Format raw data, print messages to check ^

    ############################################################################
    
    ### Perform the perturbation v
    ###

    reprogramming_df = [
        fb_df
    ]

    perturb = fb_df.copy()
    recipe = ";".join(recipe_list)
    perturb['recipe'] = recipe
    perturb['type'] = 'reprogrammed'
    perturb['input_ids'] = perturb['input_ids'].apply(lambda x: add_perturbations_to_cell(x, get_tokens_list(recipe_list)))

    reprogramming_df.append(perturb)

    reprogramming_df = pd.concat(reprogramming_df, ignore_index=True)

    print(f"{reprogramming_df.shape=}")

    ###
    ### Perform the perturbation ^

    ############################################################################
    ### Get the embeddings into an Anndata object v
    ###

    torch.cuda.empty_cache()

    reprogramming_data = Dataset.from_pandas(reprogramming_df)

    reprogramming_embs = extract_embedding_in_mem(
        model, 
        reprogramming_data, 
        layer_to_quant=-1,
        forward_batch_size=100,
    )
    print(f"{reprogramming_embs.shape=}")

    # translate into an anndata object and plot
    reprogramming_adata = embedding_to_adata(reprogramming_embs)
    reprogramming_adata.obs = reprogramming_df.copy()


    reprogramming_adata.obs.head()

    return reprogramming_adata
    ###
    ### Perform the perturbation ^


    

In [29]:
# comment out parts as you add them to main.
# don't add the globals or the prelims

def main():
    # load the model

    jobNumber = 1
    # For our iterations specifically...
    all_input_recipes = ten_choose_five()
    recipe_list = all_input_recipes.at[all_input_recipes[all_input_recipes['recipe_iteration'] == jobNumber].index[0], 'recipe_list']
    

    # run on a specific perturbation ( last thing not yet: change with ten_choose_five() )
    reprogramming_adata = single_recipe(recipe_list)


    reprogramming_adata.obs = check_and_convert(reprogramming_adata.obs)
    reprogramming_adata.var = check_and_convert(reprogramming_adata.var)
   
        
    filepath = f"/home/oliven/test_trash/jupyter_notebooks/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_job_number_{jobNumber}.h5ad"

    try:
        reprogramming_adata.write(filepath)
        print(f"File successfully written to {filepath}")
    except Exception as e:
        print(f"Error occurred: {e}")
    

    # get and then return reprogramming_anndata

    # save to a specific place ( user -> turbo. do both later)

main()



Original Dataset:
  - Number of samples: 214,715
  - Number of columns: 8

Filtered Dataset:
  - Number of samples: 10
  - Number of columns: 11

Cell Type Distribution (Filtered):
standardized_cell_type
Fibroblast    10
Name: count, dtype: int64

Dataset converted back: Dataset({
    features: ['input_ids', 'cell_type', 'dataset', 'length', 'ignore', 'standardized_cell_type', 'broad_type', '__index_level_0__', 'cell_id', 'recipe', 'type'],
    num_rows: 10
})
reprogramming_df.shape=(20, 11)


100%|██████████| 1/1 [00:00<00:00,  1.29it/s]

reprogramming_embs.shape=(20, 512)
File successfully written to /home/oliven/test_trash/jupyter_notebooks/2024-07-30_19-15-31_job_number_1.h5ad


In [ ]:
break

In [ ]:
## Testing string naming
from datetime import datetime

    # Get the current date and time
now = datetime.now()
    
    # Format the date and time
date_time_str = now.strftime("%Y-%m-%d_%H-%M-%S")


jobNumber = 10
filename = f"{date_time_str}_job_number_{jobNumber}"
filepath = '/home/oliven/test_trash' + filename + '.h5ad'
reprogramming_adata.write(filepath)

print(filepath)

In [ ]:
from datetime import datetime

# Assuming reprogramming_adata is already defined and is a valid AnnData object

# Get the current date and time
now = datetime.now()
    
# Format the date and time
date_time_str = now.strftime("%Y-%m-%d_%H-%M-%S")

# Define jobNumber
jobNumber = 10

# Create the filename with "_job_number_" between date and jobNumber
filename = f"{date_time_str}_job_number_{jobNumber}.h5ad"

# Define the filepath
filepath = f'/home/oliven/test_trash/{filename}'

# Write the AnnData object to file
reprogramming_adata.write(filepath)

print(filepath)


In [ ]:
from datetime import datetime

# Assuming reprogramming_adata is already defined and is a valid AnnData object

# Get the current date and time
now = datetime.now()
    
# Format the date and time
date_time_str = now.strftime("%Y-%m-%d_%H-%M-%S")

# Define jobNumber
jobNumber = 10

# Create the filename with "_job_number_" between date and jobNumber
filename = f"{date_time_str}_job_number_{jobNumber}.h5ad"

# Define the filepath
filepath = f'/home/oliven/test_trash/{filename}'

# Check and ensure all data is of correct type
def check_and_convert(data):
    if isinstance(data, pd.DataFrame):
        for col in data.columns:
            if not pd.api.types.is_string_dtype(data[col]):
                data[col] = data[col].astype(str)
    return data

reprogramming_adata.obs = check_and_convert(reprogramming_adata.obs)
reprogramming_adata.var = check_and_convert(reprogramming_adata.var)

# Write the AnnData object to file
try:
    reprogramming_adata.write(filepath)
    print(f"File successfully written to {filepath}")
except Exception as e:
    print(f"Error occurred: {e}")

print(filepath)


In [ ]:
break

In [ ]:
initial = reprogramming_adata.obs[reprogramming_adata.obs['type'] == 'initial']
target = reprogramming_adata.obs[reprogramming_adata.obs['type'] == 'target']
repro = reprogramming_adata.obs[reprogramming_adata.obs['type'] == 'reprogrammed']

# precompute all distances
metric = 'cosine'
D = squareform(pdist(reprogramming_adata.X, metric=metric))
print(f"{D.shape=}")

inital_to_target = D[initial.index, target.index].mean()
print(f"{inital_to_target=:.4f}")

result = []

for i, (recipe, group) in enumerate(repro.groupby('recipe')):

    # might want to 
    if i % 25 == 0:
        print(f"Recipe {i}/{len(inputs)}...")
    
    # compute group to intial
    recipe_to_initial = D[group.index, initial.index].mean() # average over all cells
    
    # compute group to target
    recipe_to_target = D[group.index, target.index].mean() # average over all cells
    
    row = {
        'recipe' : recipe,
        'recipe_to_initial' : recipe_to_initial,
        'recipe_to_target' : recipe_to_target,
        'recipe_diff' : inital_to_target - recipe_to_target,
    }
    result.append(row)
    
result = pd.DataFrame(result)
result.head(10)